In [18]:
import pandas as pd
import shutil
import zipfile
import os

from pathlib     import Path
from collections import defaultdict

In [19]:
types = {
    0: "Bass",
    1: "Brass",
    2: "Keys",
    3: "Lead",
    4: "Organ",
    5: "Pad",
    6: "Percussion",
    7: "Sequence",
    8: "SFX",
    9: "Strings"
}

In [20]:
types_short = {
    0: "BS",
    1: "BR",
    2: "KS",
    3: "LD",
    4: "OR",
    5: "PD",
    6: "DR",
    7: "SQ",
    8: "FX",
    9: "ST"
}

In [21]:
clean_dir     = Path("../patches/clean")
generated_dir = Path("../patches/generated")
temp_dir      = Path("../patches/temp")

generated_dir.mkdir(parents=True, exist_ok=True)

In [22]:
patches = []

for clean_file in clean_dir.glob("*.csv"):

    patch = pd.read_csv(clean_file)
    patch["bank_name"] = clean_file.stem

    patch = patch[[
        "bank_name",
        "patch_name",
        "patch_type",
        "path"
    ]]

    patch = patch[patch["patch_name"] != "Init"]

    patches.append(patch)

In [23]:
patches = pd.concat(patches)
patches = patches.drop_duplicates(subset=["patch_name", "patch_type"])
patches = patches.reset_index(drop=True)

In [24]:
patches["patch_type_short"]  = patches["patch_type"].map(types_short)
patches["patch_type"]        = patches["patch_type"].map(types)

In [25]:
patches[patches.patch_type == "Brass"]

,bank_name,patch_name,patch_type,path,patch_type_short
543,factory_3,Raver,Brass,factory_3/Factory3.0/01-00122021 17h55-A/283-0...,BR
559,factory_3,Strings,Brass,factory_3/Factory3.0/01-00122021 17h55-A/259-0...,BR
561,factory_3,Profet,Brass,factory_3/Factory3.0/01-00122021 17h55-A/342-0...,BR
657,factory_3,Brass,Brass,factory_3/Factory3.0/01-00122021 17h55-A/262-0...,BR
663,factory_3,Bells,Brass,factory_3/Factory3.0/01-00122021 17h55-A/278-0...,BR
675,factory_3,Classist,Brass,factory_3/Factory3.0/01-00122021 17h55-A/125-0...,BR
695,factory_3,Acid Bass,Brass,factory_3/Factory3.0/01-00122021 17h55-A/275-0...,BR
696,factory_3,Snare Seq,Brass,factory_3/Factory3.0/01-00122021 17h55-A/223-0...,BR
721,factory_3,Keys,Brass,factory_3/Factory3.0/01-00122021 17h55-A/252-0...,BR
769,factory_3,Emo,Brass,factory_3/Factory3.0/01-00122021 17h55-A/336-0...,BR


In [26]:
paths_by_type = defaultdict(list)

for _, row in patches.iterrows():
    patch_type = row["patch_type"]
    patch_path = row["path"]

    src_path = temp_dir / patch_path
    paths_by_type[patch_type].append(src_path)

In [27]:
for patch_type, paths in paths_by_type.items():
    type_dir = generated_dir / patch_type
    subfolder = type_dir / f"01-{patch_type}-A"
    subfolder.mkdir(parents=True, exist_ok=True)

    for i, src_path in enumerate(paths):
        patch_num = i + 1
        new_name = f"01-{patch_type}-A{patch_num}.mbp"
        dst_path = subfolder / new_name
        if src_path.exists():
            shutil.copy(src_path, dst_path)